In [8]:
import numpy as np
import theano
import lasagne
import random
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from sklearn.metrics import classification_report, confusion_matrix
import IOutils

In [25]:
ds = IOutils.data_streamer2()
vt = IOutils.VectorTransformer()

In [26]:
X_val, Y_val = ds.next()

In [27]:
X_val = X_val/X_val.max()

In [28]:
Y_val = vt.transform(Y_val)

In [32]:
np.sum(Y_val == 0) / float(len(Y_val)),np.sum(Y_val != 0) / float(len(Y_val))

(0.80714835643034077, 0.19285164356965923)

In [6]:
nn = NeuralNet(layers=[
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('hidden2', layers.DenseLayer),
        ('output', layers.DenseLayer)
    ],
    input_shape=(None, 42),
    hidden_num_units= 100,
    hidden2_num_units=100,
    output_num_units = len(vt.unique_rows),
    output_nonlinearity=lasagne.nonlinearities.softmax,
    update_learning_rate = 0.1,
    update = nesterov_momentum,
    max_epochs=100,
    verbose=10000)

In [9]:
X_next, Y_next = ds.next()
X_next = X_next/X_next.max()
X_next[-256:, 32:]=0
zipped = zip(X_next,Y_next)
random.shuffle(zipped)
X,Y = zip(*zipped)
X_next = np.array(X)
Y_next = np.array(Y)
X_next = X_next.astype(np.float)
Y_next = vt.transform(Y_next)
nn.fit(X_next, Y_next)

# Neural Network with 15612 learnable parameters

## Layer information

  #  name       size
---  -------  ------
  0  input        42
  1  hidden      100
  2  hidden2     100
  3  output       12

  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  -----
      1       0.41795       0.43473      0.96140      0.90655  1.67s
      2       0.37831       0.37634      1.00526      0.90694  1.59s
      3       0.35069       0.36068      0.97233      0.90737  1.48s
      4       0.32793       0.32793      1.00000      0.90831  1.44s
      5       0.31918       0.32352      0.98657      0.90815  1.46s
      6       0.30438       0.30904      0.98493      0.90911  1.76s
      7       0.29946       0.30337      0.98713      0.91047  1.97s
      8       0.28959       0.29010      0.99822      0.91159  1.93s
      9       0.27663       0.28054      0.98607      0.91214  1.89s
     10       0.26993       0.26968      1.0

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x1072b1b90>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x1072b1b10>,
     custom_score=None, hidden2_num_units=100, hidden_num_units=100,
     input_shape=(None, 42),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('hidden2', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=100, more_params={},
     objective=<function objective at 0x1072bc6e0>,
     objective_loss_function=<function categorical_crossentropy at 0x106f232a8>,
     on_batch_finished=[],
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog instance at 0x112b583f8>],
     on_training_finished=[],
     on_training_started=[<nolearn.lasagne.handlers.PrintLayerInfo instance at 0x112b58440>],
     output_nonlinearity=<function softmax at 0x1

In [10]:
predicted = nn.predict(X_val.astype(np.float))

In [11]:
confusion_matrix(Y_val, predicted)

array([[95766,     0,    36,   640,     0,     0,     0,     2,     7,
            0,     0],
       [ 2779,     0,     1,    30,     0,     0,     0,     0,     0,
            0,     0],
       [ 1443,     0,     0,     3,     0,     0,     0,     0,     0,
            0,     0],
       [ 5035,     0,     1,    64,     0,     0,     0,     0,     0,
            0,     0],
       [  341,     0,     1,     1,     0,     0,     0,     0,     0,
            0,     0],
       [  834,     0,     0,    10,     0,     0,     0,     0,     0,
            0,     0],
       [ 1351,     0,     0,     1,     0,     0,     0,     0,     0,
            0,     0],
       [  497,     0,     0,     4,     0,     0,     0,     0,     0,
            0,     0],
       [ 3113,     0,     0,    39,     0,     0,     0,     1,     0,
            0,     0],
       [ 3712,     0,     1,    35,     0,     0,     0,     0,     0,
            0,     0],
       [ 3732,     0,     0,    16,     0,     0,     0,    

In [12]:
print classification_report(Y_val, predicted)

             precision    recall  f1-score   support

          0       0.81      0.99      0.89     96451
          1       0.00      0.00      0.00      2810
          2       0.00      0.00      0.00      1446
          3       0.08      0.01      0.02      5100
          4       0.00      0.00      0.00       343
          5       0.00      0.00      0.00       844
          6       0.00      0.00      0.00      1352
          7       0.00      0.00      0.00       501
          8       0.00      0.00      0.00      3153
         10       0.00      0.00      0.00      3748
         11       0.00      0.00      0.00      3748

avg / total       0.65      0.80      0.72    119496



/Users/zaf/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
len(vt.unique_rows)

12